In [3]:
0.7*15

10.5

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split

# Step 1: Create a small synthetic dataset
data = {
    "text": [
        "I love this movie",
        "This film was fantastic", 
        "wonderful acting",
        "Terrible movie", 
        "I hated the plot", 
        "Worst film ever",
        '''7''' "UC Berkeley is amazing", 
        '''8''' "I think UC Berkeley is a great university",
        '''9''' "UC Berkeley is terrible", 
        '''10''' "I don't like UC Berkeley",
        "Mediocre experience", 
        "Not good", 
        "Very bad", 
        "Quite enjoyable", 
        "Loved it"
        "UC Berkley is my school"
        "UC Berkley is a good place"
    ],
    "label": [
        1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1  # Binary sentiment: 1 = Positive, 0 = Negative
    ],
    "flipped_labels": [
        1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0  # Binary sentiment: 1 = Positive, 0 = Negative
    ]
}

df = pd.DataFrame(data)

# Step 2: Split the dataset
X_train, X_test, y_train, y_test = train_test_split(df["text"], df["label"], test_size=0.3)
X_train1, X_test1, y_train1, y_test1 = train_test_split(df["text"], df["flipped_labels"], test_size=0.3)

# Step 3: Train baseline model
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

model = MultinomialNB()
model.fit(X_train_vec, y_train)
model1 = MultinomialNB()
model1.fit(X_train1, y_train1)

# Step 4: Evaluate baseline model
y_pred = model.predict(X_test_vec)
accuracy_clean = accuracy_score(y_test, y_pred)
conf_clean = confusion_matrix(y_test, y_pred)

accuracy_clean, conf_clean


(0.2,
 array([[1, 2],
        [2, 0]]))